# Finetune PE dataset for LI

## Libraries

In [1]:
# %cd ..
# %rm -rf LLaMA-Factory
# !git clone https://github.com/hiyouga/LLaMA-Factory.git
# %cd LLaMA-Factory
# %ls
# !pip install -e .[torch,bitsandbytes]

In [2]:
# !pip uninstall -y pydantic
# !pip install pydantic==1.10.9 # 

# !pip uninstall -y gradio
# !pip install gradio==3.48.0

# !pip uninstall -y bitsandbytes
# !pip install --upgrade bitsandbytes

# !pip install tqdm
# !pip install ipywidgets
# !pip install scikit-learn

# Restart kernel afterwards.

In [1]:
import os
import ast
import json
import torch
import pickle
import pandas as pd
from tqdm.notebook import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report

In [2]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

## Post-processing

In [3]:
output_dir = "models/PE_LI_essay_llama-3-8b-Instruct"
nb_epochs = 10

# comment in the corrected notebook

In [4]:
# %cd ..

In [5]:
with open(os.path.join(output_dir, f"""PE_LI_results_{nb_epochs}.pickle"""), "rb") as fh:
        
        results = pickle.load(fh)

In [6]:
results

{'ground_truths': ['{"list_argument_relations": [[3, 0], [3, 1], [3, 2], [3, 0], [3, 1], [3, 2], [2, 0], [2, 1]]}',
  '{"list_argument_relations": [[3, 0], [3, 1], [3, 2], [5, 0], [5, 1], [5, 2], [2, 3], [5, 4]]}',
  '{"list_argument_relations": [[4, 0], [0, 1], [4, 2], [4, 3], [4, 5], [4, 0], [4, 1], [4, 2], [4, 3], [4, 5], [4, 6]]}',
  '{"list_argument_relations": [[2, 0], [0, 1], [2, 3], [2, 0], [2, 1]]}',
  '{"list_argument_relations": [[1, 0], [1, 2], [4, 3], [4, 5], [4, 6], [2, 0], [2, 1], [6, 2], [6, 3], [3, 4], [3, 5], [0, 1], [0, 2], [0, 3], [0, 4]]}',
  '{"list_argument_relations": [[2, 0], [0, 1], [0, 1], [0, 2], [0, 3], [3, 0], [2, 1], [3, 2], [3, 4]]}',
  '{"list_argument_relations": [[5, 0], [2, 1], [5, 2], [4, 3], [5, 4], [0, 1], [0, 2], [2, 1], [0, 2], [0, 3]]}',
  '{"list_argument_relations": [[0, 1], [0, 1]]}',
  '{"list_argument_relations": [[0, 1], [3, 2], [0, 3], [1, 0], [2, 0], [2, 1]]}',
  '{"list_argument_relations": [[0, 1], [0, 1], [3, 2], [0, 3], [0, 4], [4, 

In [7]:
grounds = results["ground_truths"]

In [8]:
preds = results["predictions"]

In [9]:
grounds = [json.loads(x)["list_argument_relations"] for x in grounds]

In [10]:
preds = [x["content"] for x in preds]

In [11]:
preds = [json.loads(x)["list_argument_relations"] for x in preds]

In [12]:
len(grounds), len(preds)

(80, 80)

In [13]:
grounds

[[[3, 0], [3, 1], [3, 2], [3, 0], [3, 1], [3, 2], [2, 0], [2, 1]],
 [[3, 0], [3, 1], [3, 2], [5, 0], [5, 1], [5, 2], [2, 3], [5, 4]],
 [[4, 0],
  [0, 1],
  [4, 2],
  [4, 3],
  [4, 5],
  [4, 0],
  [4, 1],
  [4, 2],
  [4, 3],
  [4, 5],
  [4, 6]],
 [[2, 0], [0, 1], [2, 3], [2, 0], [2, 1]],
 [[1, 0],
  [1, 2],
  [4, 3],
  [4, 5],
  [4, 6],
  [2, 0],
  [2, 1],
  [6, 2],
  [6, 3],
  [3, 4],
  [3, 5],
  [0, 1],
  [0, 2],
  [0, 3],
  [0, 4]],
 [[2, 0], [0, 1], [0, 1], [0, 2], [0, 3], [3, 0], [2, 1], [3, 2], [3, 4]],
 [[5, 0],
  [2, 1],
  [5, 2],
  [4, 3],
  [5, 4],
  [0, 1],
  [0, 2],
  [2, 1],
  [0, 2],
  [0, 3]],
 [[0, 1], [0, 1]],
 [[0, 1], [3, 2], [0, 3], [1, 0], [2, 0], [2, 1]],
 [[0, 1], [0, 1], [3, 2], [0, 3], [0, 4], [4, 5], [4, 6]],
 [[0, 1],
  [0, 2],
  [0, 3],
  [0, 1],
  [0, 2],
  [0, 3],
  [0, 4],
  [0, 5],
  [0, 6],
  [4, 0],
  [4, 1],
  [4, 2],
  [2, 3]],
 [[2, 0], [2, 1], [0, 1], [1, 2], [0, 3], [3, 4], [0, 5], [5, 6]],
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 1], [0, 2], [0, 3], 

In [14]:
preds

[[[3, 0], [3, 1], [3, 2], [3, 0], [3, 1], [3, 2], [2, 0], [2, 1]],
 [[3, 0], [0, 1], [3, 2], [5, 0], [5, 1], [5, 2], [2, 3], [5, 4]],
 [[4, 0],
  [4, 1],
  [4, 2],
  [4, 3],
  [4, 5],
  [4, 0],
  [4, 1],
  [4, 2],
  [4, 3],
  [4, 5],
  [4, 6]],
 [[2, 0], [2, 1], [2, 3], [2, 0], [2, 1]],
 [[1, 0],
  [1, 2],
  [4, 3],
  [4, 5],
  [4, 6],
  [6, 0],
  [2, 1],
  [6, 2],
  [4, 3],
  [6, 4],
  [6, 5],
  [0, 1],
  [0, 2],
  [0, 3],
  [0, 4]],
 [[2, 0], [2, 1], [0, 1], [0, 2], [0, 3], [3, 0], [2, 1], [3, 2], [3, 4]],
 [[5, 0],
  [2, 1],
  [4, 2],
  [4, 3],
  [5, 4],
  [0, 1],
  [1, 2],
  [2, 1],
  [0, 2],
  [0, 3]],
 [[0, 1], [0, 1]],
 [[0, 1], [3, 2], [0, 3], [1, 0], [2, 0], [2, 1]],
 [[0, 1], [0, 1], [3, 2], [0, 3], [0, 4], [0, 5], [0, 6]],
 [[0, 1],
  [0, 2],
  [0, 3],
  [0, 1],
  [0, 2],
  [0, 3],
  [0, 4],
  [0, 5],
  [0, 6],
  [4, 0],
  [4, 1],
  [4, 2],
  [4, 3]],
 [[2, 0], [2, 1], [2, 1], [0, 2], [0, 3], [3, 4], [0, 5], [5, 6]],
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 1], [0, 2], [0, 3], 

In [16]:
for x,y in zip(grounds, preds):
    if len(x) != len(y):
        print(x, y)

[[1, 2], [2, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 1]] [[0, 1], [1, 2], [1, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 1]]
[[0, 1], [3, 2], [0, 3], [5, 4], [0, 5], [5, 6], [6, 7], [2, 1], [0, 2], [4, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8]] [[0, 1], [3, 2], [0, 3], [0, 4], [0, 5], [0, 6], [6, 7], [0, 1], [1, 2], [4, 3], [0, 4], [0, 5], [0, 6], [6, 7], [6, 8], [6, 9]]
[[0, 1], [0, 2], [1, 0]] [[0, 1], [0, 2], [1, 0], [1, 2]]


In [17]:
pe_df = pd.read_csv(os.path.abspath("datasets/PE_data.csv"))

In [18]:
# pe_df

In [19]:
df_split = pd.read_csv("datasets/train-test-split.csv", sep=";")

In [20]:
pe_df['split'] = pe_df['essay_id'].map(df_split['SET'])

In [21]:
def get_ac_count(x):

    return len(ast.literal_eval(x.AC_types))

In [22]:
pe_df["AC_count"] = pe_df.apply(lambda x: get_ac_count(x), axis=1)

In [26]:
pe_df

,essay_id,para_id,para_types,para_text,adu_spans,ac_spans,ai_spans,AC_types,AR_pairs,AR_types,split,AC_count,LI_grounds
0,0,0,prompt,<prompt> Should students be taught to compete ...,[],[],[],[],[],[],TRAIN,0,[]
1,0,1,intro,<para-intro> It is always said that competitio...,"[(76, 97)]","[(86, 97)]","[(76, 85)]",['MajorClaim'],[],[],TRAIN,1,[]
2,0,2,body,"<para-body> First of all , <AC> through cooper...","[(1, 25), (26, 55), (56, 99), (100, 123)]","[(5, 25), (27, 55), (57, 99), (101, 123)]","[(1, 4), (26, 26), (56, 56), (100, 100)]","['Claim', 'Premise', 'Premise', 'Premise']","[(0, 1), (0, 2), (0, 3)]","['Support', 'Support', 'Support']",TRAIN,4,"[[0, 1, Rel], [0, 2, Rel], [0, 3, Rel], [1, 0,..."
3,0,3,body,"<para-body> On the other hand , <AC> the signi...","[(1, 22), (24, 37), (39, 63), (76, 139), (155,...","[(6, 22), (30, 37), (41, 63), (77, 139), (156,...","[(1, 5), (24, 29), (39, 40), (76, 76), (155, 1...","['Premise', 'Claim', 'Premise', 'Premise', 'Cl...","[(1, 0), (4, 2), (4, 3)]","['Support', 'Support', 'Support']",TRAIN,5,"[[0, 1, N-Rel], [0, 2, N-Rel], [0, 3, N-Rel], ..."
4,0,4,conclusion,"<para-conclusion> Consequently , no matter fro...","[(1, 40)]","[(25, 40)]","[(1, 24)]",['MajorClaim'],[],[],TRAIN,1,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,401,2230,prompt,<prompt> A greater proportion of the budget sh...,[],[],[],[],[],[],TRAIN,0,[]
2231,401,2231,intro,"<para-intro> In today ' s world , the concept ...","[(26, 51)]","[(33, 51)]","[(26, 32)]",['MajorClaim'],[],[],TRAIN,1,[]
2232,401,2232,body,<para-body> <AC> The first reason why educatio...,"[(0, 22), (24, 56), (58, 71), (72, 98)]","[(1, 22), (27, 56), (60, 71), (74, 98)]","[(0, 0), (24, 26), (58, 59), (72, 73)]","['Premise', 'Premise', 'Claim', 'Premise']","[(2, 0), (2, 1), (2, 3)]","['Support', 'Support', 'Support']",TRAIN,4,"[[0, 1, N-Rel], [0, 2, N-Rel], [0, 3, N-Rel], ..."
2233,401,2233,body,<para-body> The second reason why <AC> governm...,"[(1, 25), (27, 53), (55, 88), (90, 128)]","[(5, 25), (31, 53), (57, 88), (93, 128)]","[(1, 4), (27, 30), (55, 56), (90, 92)]","['Claim', 'Premise', 'Premise', 'Premise']","[(2, 1), (0, 2), (2, 3)]","['Support', 'Support', 'Support']",TRAIN,4,"[[0, 1, N-Rel], [0, 2, Rel], [0, 3, N-Rel], [1..."


In [23]:
def get_all_possible_pairs(x):

    pairs_l = []

    ac_count = x.AC_count
    ar_pairs = ast.literal_eval(x.AR_pairs)
    
    for i in range(ac_count):
        for j in range(ac_count):
            if i != j:
                pair = (i, j)
                if pair in ar_pairs:
                    pairs_l.append([i, j, "Rel"])
                else:
                    pairs_l.append([i, j, "N-Rel"])

    return pairs_l

In [24]:
pe_df["LI_grounds"] = pe_df.apply(lambda x: get_all_possible_pairs(x), axis=1)

In [25]:
# pe_df

In [25]:
LI_grounds_l = list(pe_df[pe_df.split == "TEST"].reset_index().LI_grounds)

In [26]:
len(LI_grounds_l), len(preds)

(439, 80)

In [27]:
LI_grounds_l

[[],
 [],
 [[0, 1, 'N-Rel'],
  [0, 2, 'N-Rel'],
  [0, 3, 'N-Rel'],
  [1, 0, 'N-Rel'],
  [1, 2, 'N-Rel'],
  [1, 3, 'N-Rel'],
  [2, 0, 'N-Rel'],
  [2, 1, 'N-Rel'],
  [2, 3, 'N-Rel'],
  [3, 0, 'Rel'],
  [3, 1, 'Rel'],
  [3, 2, 'Rel']],
 [[0, 1, 'N-Rel'],
  [0, 2, 'N-Rel'],
  [0, 3, 'N-Rel'],
  [1, 0, 'N-Rel'],
  [1, 2, 'N-Rel'],
  [1, 3, 'N-Rel'],
  [2, 0, 'N-Rel'],
  [2, 1, 'N-Rel'],
  [2, 3, 'N-Rel'],
  [3, 0, 'Rel'],
  [3, 1, 'Rel'],
  [3, 2, 'Rel']],
 [[0, 1, 'N-Rel'],
  [0, 2, 'N-Rel'],
  [1, 0, 'N-Rel'],
  [1, 2, 'N-Rel'],
  [2, 0, 'Rel'],
  [2, 1, 'Rel']],
 [],
 [],
 [],
 [[0, 1, 'N-Rel'],
  [0, 2, 'N-Rel'],
  [0, 3, 'N-Rel'],
  [1, 0, 'N-Rel'],
  [1, 2, 'N-Rel'],
  [1, 3, 'N-Rel'],
  [2, 0, 'N-Rel'],
  [2, 1, 'N-Rel'],
  [2, 3, 'N-Rel'],
  [3, 0, 'Rel'],
  [3, 1, 'Rel'],
  [3, 2, 'Rel']],
 [[0, 1, 'N-Rel'],
  [0, 2, 'N-Rel'],
  [0, 3, 'N-Rel'],
  [0, 4, 'N-Rel'],
  [0, 5, 'N-Rel'],
  [1, 0, 'N-Rel'],
  [1, 2, 'N-Rel'],
  [1, 3, 'N-Rel'],
  [1, 4, 'N-Rel'],
  [1, 5, 'N-Rel'],
  [2,

In [28]:
preds

[[[3, 0], [3, 1], [3, 2], [3, 0], [3, 1], [3, 2], [2, 0], [2, 1]],
 [[3, 0], [0, 1], [3, 2], [5, 0], [5, 1], [5, 2], [2, 3], [5, 4]],
 [[4, 0],
  [4, 1],
  [4, 2],
  [4, 3],
  [4, 5],
  [4, 0],
  [4, 1],
  [4, 2],
  [4, 3],
  [4, 5],
  [4, 6]],
 [[2, 0], [2, 1], [2, 3], [2, 0], [2, 1]],
 [[1, 0],
  [1, 2],
  [4, 3],
  [4, 5],
  [4, 6],
  [6, 0],
  [2, 1],
  [6, 2],
  [4, 3],
  [6, 4],
  [6, 5],
  [0, 1],
  [0, 2],
  [0, 3],
  [0, 4]],
 [[2, 0], [2, 1], [0, 1], [0, 2], [0, 3], [3, 0], [2, 1], [3, 2], [3, 4]],
 [[5, 0],
  [2, 1],
  [4, 2],
  [4, 3],
  [5, 4],
  [0, 1],
  [1, 2],
  [2, 1],
  [0, 2],
  [0, 3]],
 [[0, 1], [0, 1]],
 [[0, 1], [3, 2], [0, 3], [1, 0], [2, 0], [2, 1]],
 [[0, 1], [0, 1], [3, 2], [0, 3], [0, 4], [0, 5], [0, 6]],
 [[0, 1],
  [0, 2],
  [0, 3],
  [0, 1],
  [0, 2],
  [0, 3],
  [0, 4],
  [0, 5],
  [0, 6],
  [4, 0],
  [4, 1],
  [4, 2],
  [4, 3]],
 [[2, 0], [2, 1], [2, 1], [0, 2], [0, 3], [3, 4], [0, 5], [5, 6]],
 [[0, 1], [0, 2], [0, 3], [0, 4], [0, 1], [0, 2], [0, 3], 

In [ ]:
def process_preds(test_preds, LI_grounds_l):

    if len(test_preds) != len(LI_grounds_l):
        print("error in preds length")

    preds_pairs_l = []

    for l_1, l_2 in zip(test_preds, LI_grounds_l):
        para_list = []
        for i, j, _ in l_2:
            pair = [i, j]
            if pair in l_1:
                para_list.append([i, j, "Rel"])
            else:
                para_list.append([i, j, "N-Rel"])

        preds_pairs_l.append(para_list)

    return preds_pairs_l

In [ ]:
LI_preds_l = process_preds(preds, LI_grounds_l)

In [ ]:
# sanity check: 
len(LI_grounds_l) == len(LI_preds_l)

In [ ]:
LI_preds = [item for row in LI_preds_l for item in row]
LI_grounds = [item for row in LI_grounds_l for item in row]

In [ ]:
# sanity check: 
len(LI_preds) == len(LI_grounds)

In [ ]:
LI_preds = [x[2] for x in LI_preds]
LI_grounds = [x[2] for x in LI_grounds]

In [ ]:
## Results

In [ ]:
print(classification_report(LI_grounds, LI_preds, digits=3))

In [ ]:
with open(f"""{output_dir}/classification_report.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(LI_grounds, LI_preds, output_dict=True), fh)

In [ ]:
del model

In [ ]:
torch.cuda.empty_cache()